## chrXY Assembly Evaluation

JM 12/14/2021
Google Drive location with README summarizing work: https://drive.google.com/drive/folders/1LqG1aBxW8pvTAB0yX2S3BnV2SZLmLzTn?usp=sharing

## Purpose:
JZ would like to evaluate the most recent chr X and Y assemblies from T2T-polishing.  It is hoped that these will be used for eventual assembly-based benchmarks for chrX and Y. Versions have polished to remove most small errors but not SVs yet. 

# <font color='red'>T2T HG002 chrXY assembly v2.1</font>

**assembly v2.1 from**\ 
https://s3-us-west-2.amazonaws.com/human-pangenomics/working/T2T/HG002XY/v2.1/v2.1.XY.fasta

### <font color='blue'>Step 1: Prepare T2T chrXY assembly for use as benchmark</font>
1. split assembly by chromosome
2. call variants 

In [3]:
#Downloaded assembly from AWS using command:\
aws s3 cp s3://human-pangenomics/working/T2T/HG002XY/v2.1/v2.1.XY.fasta T2T-chrXY-assembly/

download: s3://human-pangenomics/working/T2T/HG002XY/v2.1/v2.1.XY.fasta to T2T-chrXY-assembly/v2.1.XY.fasta


In [7]:
#split XY fasta to chrX and chrY
awk '/^>/{f="chr."++d} {print > f}' T2T-chrXY-assembly/v2.1.XY.fasta
mv chr.1 T2T-chrXY-assembly/chrX.fa
mv chr.2 T2T-chrXY-assembly/chrY.fa

In [11]:
#check that split fasta have same total lines as complete fasta. Confirm headers are correct. 
wc -l T2T-chrXY-assembly/chrX.fa
wc -l T2T-chrXY-assembly/chrY.fa
wc -l T2T-chrXY-assembly/v2.1.XY.fasta
grep '>' T2T-chrXY-assembly/chrX.fa
grep '>' T2T-chrXY-assembly/chrY.fa

 2572398 T2T-chrXY-assembly/chrX.fa
 1040949 T2T-chrXY-assembly/chrY.fa
 3613347 T2T-chrXY-assembly/v2.1.XY.fasta
>chrX_hg002
>chrY_hg002


In [55]:
#Run Dipcall v0.3 in Docker container aligning to GRCh37 and GRCh38. 
caffeinate -s sh dipcall/dipcall-T2T-chrXY.sh

Wed Dec 15 09:36:23 EST 2021
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/chrY.fa 2> /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap1.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap1.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/chrX.fa 2> /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap2.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap2.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -a -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/chrY.fa 2> /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap1.sam.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh38.hap1.sam.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -a -xasm5 --cs -t8 -z

/data/dipcall/dipcall.kit_0.3/k8 /data/dipcall/dipcall.kit_0.3/dipcall-aux.js samflt /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.hap1.sam.gz | /data/dipcall/dipcall.kit_0.3/samtools sort -m4G --threads 4 -o /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.hap1.bam -
/data/dipcall/dipcall.kit_0.3/htsbox pileup -q5 -evcf /data/dipcall/references/GRCh37.fa /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.hap1.bam /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.hap2.bam | /data/dipcall/dipcall.kit_0.3/htsbox bgzip > /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.pair.vcf.gz
/data/dipcall/dipcall.kit_0.3/k8 /data/dipcall/dipcall.kit_0.3/dipcall-aux.js vcfpair  -p /data/dipcall/dipcall.kit_0.3/hs37d5.PAR.bed /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.pair.vcf.gz | /data/dipcall/dipcall.kit_0.3/htsbox bgzip > /data/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.dip.vcf.gz
Wed Dec 15 11:06:05 EST 2021


### <font color='blue'>Step 2 : Small Variant Benchmarking against T2T-v2.1-XY-align2-GRCh38.dip.vcf with hap.py</font>
1. Query = Hifi-DV (from Google) -- no target regions
2. Query = HG002-cur.20211005 (HPRC) -- target regions = HG002-cur.20211005.dip.bed
3. hap.py will be run with v3.12 and new version 3.15 for comparison.

In [ ]:
#Get Query callsets

#Hifi-DV callset
wget -P query-callsets/ https://data.nist.gov/od/ds/ark:/88434/mds2-2336/submission_vcfs/B1S5A/B1S5A_HG002.vcf.gz

#HG002-cur.20211005.vcf.gz downloaded from https://drive.google.com/file/d/1O0D9JV-gTrZCxtNFRQdbhbpGC0nqZpPY/view?usp=sharing

#### Hap.py Run #1
run hap.py benchmarking in v3.12 and v3.15 docker containers from shell script.  Notes for building v3.15 docker container in `docker_container_build/v3.15_docker_container_build_notes.txt`. 
hap.py runs will be compared.

**Results:**

No difference between v3.12 and 3.15.  DV had signicant number of FP.gt likely due to difference in how DV represents calls as 1/1 in non-PAR, while dipcall represents these as .|1 or 1|..  See Hap.py Run # 2 for followup.

In [2]:
caffeinate -s sh happy/happy.sh

start targeted hap.py v3.15 for HPRC HG002-cur.20211005 on date
2021-12-15 18:46:31,221 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         220557
[I] Non-reference VCF records: 168572
[W] overlapping records at chr1:977279 for sample 0
[W] Variants that overlap on the reference allele: 1219
[I] Total VCF records:         5820577
[I] Non-reference VCF records: 5367388
2021-12-15 18:46:57,505 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,505 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,505 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,505 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2021-12-15 

2021-12-15 18:46:57,515 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,515 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,515 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,515 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,515 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,516 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,516 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,516 WARNING  [W] too many AD fields at chrX:515818 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,516 WARNING  [W] too many AD fields at chrX:515929 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,517 WARNING  [W] too many AD fields at chrX:522301 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,517 WARNING  [W] too many AD f

2021-12-15 18:46:57,527 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,527 WARNING  [W] too many AD fields at chrX:1170088 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,527 WARNING  [W] too many AD fields at chrX:1174060 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,527 WARNING  [W] too many AD fields at chrX:1176328 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1178157 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1178158 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1178184 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1236199 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1236212 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too many AD fields at chrX:1236218 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,528 WARNING  [W] too

2021-12-15 18:46:57,536 WARNING  [W] too many AD fields at chrX:1682200 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,536 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,536 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,537 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,537 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,537 WARNING  [W] too many AD fields at chrX:1763599 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,537 WARNING  [W] too many AD fields at chrX:1763601 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,538 WARNING  [W] too many AD fields at chrX:1763704 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,538 WARNING  [W] too many AD fields at chrX:1763744 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,538 WARNING  [W] too many AD fields at chrX:1763750 max_ad = 2 retrieved: 3
2021-12-15 18:46:57,538 WARNING  [W] too

2021-12-15 18:46:58,363 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137430 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137431 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137432 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,364 WARNING  [W] too many AD fields at chrX:2137433 max_ad = 2 retrieved: 3
2021-12-15 18:46:58,365 WARNING  [W] too

...

2021-12-15 18:54:53,151 WARNING  [W] too many AD fields at chrX:628696 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:655607 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:656860 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:656861 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:693404 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:702242 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,152 WARNING  [W] too many AD fields at chrX:724945 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,153 WARNING  [W] too many AD fields at chrX:729411 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,153 WARNING  [W] too many AD fields at chrX:765698 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,153 WARNING  [W] too many AD fields at chrX:770985 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,153 WARNING  [W] too many AD f

2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1314659 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1314734 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1348643 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1352889 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1352919 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,167 WARNING  [W] too many AD fields at chrX:1353641 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,168 WARNING  [W] too many AD fields at chrX:1353645 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,168 WARNING  [W] too many AD fields at chrX:1353652 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,168 WARNING  [W] too many AD fields at chrX:1353663 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,168 WARNING  [W] too many AD fields at chrX:1370884 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,168 WARNING  [W] too

2021-12-15 18:54:53,178 WARNING  [W] too many AD fields at chrX:1764860 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1764884 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1764887 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1764890 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1773148 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1773294 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1773305 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1773355 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,179 WARNING  [W] too many AD fields at chrX:1789626 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,180 WARNING  [W] too many AD fields at chrX:1792286 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,180 WARNING  [W] too

2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137442 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137443 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137444 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137445 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137446 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137447 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137448 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,947 WARNING  [W] too many AD fields at chrX:2137449 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,948 WARNING  [W] too many AD fields at chrX:2137450 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,948 WARNING  [W] too many AD fields at chrX:2137451 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,948 WARNING  [W] too

2021-12-15 18:54:53,960 WARNING  [W] too many AD fields at chrX:2330072 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,960 WARNING  [W] too many AD fields at chrX:2330076 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2330078 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2330090 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2436499 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2516501 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2516502 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2520353 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,961 WARNING  [W] too many AD fields at chrX:2600800 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,962 WARNING  [W] too many AD fields at chrX:2685495 max_ad = 2 retrieved: 3
2021-12-15 18:54:53,962 WARNING  [W] too

In [4]:
#copy happy extended.csv files to summary_metrics for R
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.15/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.1XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.15/
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.1XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.15/non-targeted/HG002-HifiDV-against-T2Tv2.1XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.15/
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/non-targeted/HG002-HifiDV-against-T2Tv2.1XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/

**Ran each extended.csv through Rscipt `happy/summary_metrics/summary_metrics-benchmarking_against_T2T-v2.1-XY.R` to generate summary metrics. All metrics combined in excel sheet "3.12_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv"**

In [5]:
#compare md5s of output csvs of hap.py 3.12 and 3.15 to see if different. They are the same. 
md5 happy/summary_metrics/3.12_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv
md5 happy/summary_metrics/3.15_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv

MD5 (happy/summary_metrics/3.12_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv) = 2865b287570f79ca119502732a8064de
MD5 (happy/summary_metrics/3.15_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv) = 2865b287570f79ca119502732a8064de


#### Hap.py Run #2
Metrics for DV were poor, high FP.gt.  DV represents calls as 1/1 in non-PAR, while dipcall represents these as .|1 or 1|.. Hap.py has --gender flag that might fix the issue. Re-run both callsets with gender = male for comparision with run without.  Will continue using hap.py v3.12. Note: accidentally reran this in another block on 12/16)

**Results:**

Same result as Hap.py Run #1. JZ decided to change .|1 and 1|. calls to 1/1 in dipcall truth set using sed command.  See Hap.py Run #3 for followup.

In [1]:
# query = HPRC HG002-cur.20211005
caffeinate -s sh happy/happy.sh

start targeted hap.py v3.12 (gender = male) for HPRC HG002-cur.20211005 on date
2021-12-16 13:39:12,412 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         220557
[I] Non-reference VCF records: 168572
2021-12-16 13:39:15,309 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,309 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,309 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,310 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,310 WARNING  [W] too many AD fields at chrX:250410 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,310 WARNING  [W] too many AD fields at chrX:250413 max_ad = 2 retr

2021-12-16 13:39:15,324 WARNING  [W] too many AD fields at chrX:505629 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,324 WARNING  [W] too many AD fields at chrX:505639 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,325 WARNING  [W] too many AD fields at chrX:505651 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,325 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,325 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,325 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,326 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,326 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,326 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,326 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,326 WARNING  [W] too many AD f

2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1166310 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1168933 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1168998 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1169134 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1169210 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,340 WARNING  [W] too many AD fields at chrX:1169248 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,341 WARNING  [W] too many AD fields at chrX:1169269 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,341 WARNING  [W] too many AD fields at chrX:1169277 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,341 WARNING  [W] too many AD fields at chrX:1169527 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,341 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,341 WARNING  [W] too

2021-12-16 13:39:15,355 WARNING  [W] too many AD fields at chrX:1669029 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,355 WARNING  [W] too many AD fields at chrX:1669050 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,355 WARNING  [W] too many AD fields at chrX:1669089 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,355 WARNING  [W] too many AD fields at chrX:1669097 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,355 WARNING  [W] too many AD fields at chrX:1669142 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,356 WARNING  [W] too many AD fields at chrX:1678251 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,356 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,356 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,356 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,356 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2021-12-16 13:39:15,357 WARNING  [W] too

2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137420 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137421 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137422 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137423 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,124 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,125 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,125 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,125 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,125 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2021-12-16 13:39:16,125 WARNING  [W] too

...

2021-12-16 13:39:48,462 WARNING  [W] too many AD fields at chrX:366403 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366424 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366464 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366469 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366491 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366565 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,463 WARNING  [W] too many AD fields at chrX:366567 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,464 WARNING  [W] too many AD fields at chrX:366568 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,464 WARNING  [W] too many AD fields at chrX:366577 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,464 WARNING  [W] too many AD fields at chrX:366589 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,464 WARNING  [W] too many AD f

2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048534 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048538 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048540 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048711 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048715 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,472 WARNING  [W] too many AD fields at chrX:1048718 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,473 WARNING  [W] too many AD fields at chrX:1048766 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,473 WARNING  [W] too many AD fields at chrX:1050002 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,473 WARNING  [W] too many AD fields at chrX:1083936 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,473 WARNING  [W] too many AD fields at chrX:1083942 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,473 WARNING  [W] too

2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1587702 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1596694 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1596854 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1636938 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1636941 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,482 WARNING  [W] too many AD fields at chrX:1637011 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,483 WARNING  [W] too many AD fields at chrX:1639439 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,483 WARNING  [W] too many AD fields at chrX:1639516 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,483 WARNING  [W] too many AD fields at chrX:1640567 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,483 WARNING  [W] too many AD fields at chrX:1640655 max_ad = 2 retrieved: 3
2021-12-16 13:39:48,483 WARNING  [W] too

2021-12-16 13:39:49,319 WARNING  [W] too many AD fields at chrX:2137391 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,319 WARNING  [W] too many AD fields at chrX:2137392 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137393 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137394 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137395 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137396 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137397 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137398 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137399 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too many AD fields at chrX:2137400 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,320 WARNING  [W] too

2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137480 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137481 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137482 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137483 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137484 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137485 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137486 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,330 WARNING  [W] too many AD fields at chrX:2137487 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,331 WARNING  [W] too many AD fields at chrX:2137488 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,331 WARNING  [W] too many AD fields at chrX:2137506 max_ad = 2 retrieved: 3
2021-12-16 13:39:49,331 WARNING  [W] too

In [2]:
# query = HifiDV
caffeinate -s sh happy/happy.sh

start non-targeted hap.py v3.12 (gender = male) HG002-HifiDV on date
2021-12-16 13:43:49,280 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         220557
[I] Non-reference VCF records: 168572
2021-12-16 13:43:52,201 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,202 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,202 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,202 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,202 WARNING  [W] too many AD fields at chrX:250410 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,202 WARNING  [W] too many AD fields at chrX:250413 max_ad = 2 retrieved: 3
20

2021-12-16 13:43:52,211 WARNING  [W] too many AD fields at chrX:505629 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,211 WARNING  [W] too many AD fields at chrX:505639 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,211 WARNING  [W] too many AD fields at chrX:505651 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,212 WARNING  [W] too many AD f

2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1166310 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1168933 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1168998 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169134 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169210 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169248 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169269 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169277 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,222 WARNING  [W] too many AD fields at chrX:1169527 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,223 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,223 WARNING  [W] too

2021-12-16 13:43:52,238 WARNING  [W] too many AD fields at chrX:1669029 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,238 WARNING  [W] too many AD fields at chrX:1669050 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,238 WARNING  [W] too many AD fields at chrX:1669089 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,238 WARNING  [W] too many AD fields at chrX:1669097 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,239 WARNING  [W] too many AD fields at chrX:1669142 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,239 WARNING  [W] too many AD fields at chrX:1678251 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,239 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,239 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,239 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,240 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2021-12-16 13:43:52,240 WARNING  [W] too

2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137420 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137421 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137422 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137423 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,003 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,004 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,004 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,004 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,004 WARNING  [W] too

2021-12-16 13:43:53,013 WARNING  [W] too many AD fields at chrX:2228072 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,013 WARNING  [W] too many AD fields at chrX:2228189 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,013 WARNING  [W] too many AD fields at chrX:2228218 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,013 WARNING  [W] too many AD fields at chrX:2243787 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,013 WARNING  [W] too many AD fields at chrX:2276896 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too many AD fields at chrX:2277153 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too many AD fields at chrX:2277248 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too many AD fields at chrX:2277298 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too many AD fields at chrX:2277341 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too many AD fields at chrX:2278921 max_ad = 2 retrieved: 3
2021-12-16 13:43:53,014 WARNING  [W] too

In [5]:
cp -p ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/non-targeted/HG002-HifiDV-against-T2Tv2.1XY_male.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/
cp -p ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.1XY_male.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/

#### Hap.py Run #3
Use of gender flag did not solve issue with how gt were represented and handled.  JZ decided to manually change 
.|1 and 1|. calls to 1/1 in truth vcf using command 
`gunzip -c /Users/jzook/Downloads/T2T-v2.1-XY-align2-GRCh38.dip.vcf.gz | sed 's/\.|1/1\/1/;s/1|\./1\/1/' | /Applications/bioinfo/htslib-1.3/bgzip -c > /Users/jzook/Downloads/T2T-v2.1-XY-align2-GRCh38.dip_halfcalltohomvar.vcf.gz`
This will adjust the gt such that the dipcall representation now matches the DV representation. 

**Results:**
Adjusting GT dipcall representation in dicpall appears to have "improved" HifiDV but flipped HPRC FP.gt into the 20K range.

In [4]:
# query = HPRC HG002-cur.20211005
caffeinate -s sh happy/happy.sh

start targeted hap.py v3.12 (gender = male) for HPRC HG002-cur.20211005 on date
2021-12-16 13:49:58,052 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         220557
[I] Non-reference VCF records: 168572
2021-12-16 13:50:00,972 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,973 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,973 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,973 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,973 WARNING  [W] too many AD fields at chrX:250410 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,973 WARNING  [W] too many AD fields at chrX:250413 max_ad = 2 retr

2021-12-16 13:50:00,981 WARNING  [W] too many AD fields at chrX:505629 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,981 WARNING  [W] too many AD fields at chrX:505639 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,981 WARNING  [W] too many AD fields at chrX:505651 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,981 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,982 WARNING  [W] too many AD f

2021-12-16 13:50:00,996 WARNING  [W] too many AD fields at chrX:1166310 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,996 WARNING  [W] too many AD fields at chrX:1168933 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,996 WARNING  [W] too many AD fields at chrX:1168998 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,996 WARNING  [W] too many AD fields at chrX:1169134 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169210 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169248 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169269 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169277 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169527 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2021-12-16 13:50:00,997 WARNING  [W] too

2021-12-16 13:50:01,008 WARNING  [W] too many AD fields at chrX:1669029 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,008 WARNING  [W] too many AD fields at chrX:1669050 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1669089 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1669097 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1669142 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1678251 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,009 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,010 WARNING  [W] too

2021-12-16 13:50:01,776 WARNING  [W] too many AD fields at chrX:2137420 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,776 WARNING  [W] too many AD fields at chrX:2137421 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,776 WARNING  [W] too many AD fields at chrX:2137422 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,776 WARNING  [W] too many AD fields at chrX:2137423 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,776 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,777 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,777 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,777 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,777 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,777 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2021-12-16 13:50:01,778 WARNING  [W] too

...

2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366403 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366424 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366464 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366469 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366491 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,323 WARNING  [W] too many AD fields at chrX:366565 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,324 WARNING  [W] too many AD fields at chrX:366567 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,324 WARNING  [W] too many AD fields at chrX:366568 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,324 WARNING  [W] too many AD fields at chrX:366577 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,324 WARNING  [W] too many AD fields at chrX:366589 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,324 WARNING  [W] too many AD f

2021-12-16 13:50:33,331 WARNING  [W] too many AD fields at chrX:1048534 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,331 WARNING  [W] too many AD fields at chrX:1048538 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,331 WARNING  [W] too many AD fields at chrX:1048540 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1048711 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1048715 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1048718 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1048766 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1050002 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1083936 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,332 WARNING  [W] too many AD fields at chrX:1083942 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,333 WARNING  [W] too

2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1587702 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1596694 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1596854 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1636938 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1636941 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1637011 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,342 WARNING  [W] too many AD fields at chrX:1639439 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,343 WARNING  [W] too many AD fields at chrX:1639516 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,343 WARNING  [W] too many AD fields at chrX:1640567 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,343 WARNING  [W] too many AD fields at chrX:1640655 max_ad = 2 retrieved: 3
2021-12-16 13:50:33,343 WARNING  [W] too

2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137391 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137392 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137393 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137394 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137395 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,241 WARNING  [W] too many AD fields at chrX:2137396 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,242 WARNING  [W] too many AD fields at chrX:2137397 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,242 WARNING  [W] too many AD fields at chrX:2137398 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,242 WARNING  [W] too many AD fields at chrX:2137399 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,242 WARNING  [W] too many AD fields at chrX:2137400 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,242 WARNING  [W] too

2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137480 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137481 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137482 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137483 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137484 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137485 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137486 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137487 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137488 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,249 WARNING  [W] too many AD fields at chrX:2137506 max_ad = 2 retrieved: 3
2021-12-16 13:50:34,250 WARNING  [W] too

In [6]:
# query = HifiDV
caffeinate -s sh happy/happy.sh

start non-targeted hap.py v3.12 (gender = male) HG002-HifiDV on date
2021-12-16 14:11:01,816 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         220557
[I] Non-reference VCF records: 168572
2021-12-16 14:11:04,687 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,687 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,688 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,688 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,689 WARNING  [W] too many AD fields at chrX:250410 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,689 WARNING  [W] too many AD fields at chrX:250413 max_ad = 2 retrieved: 3
20

2021-12-16 14:11:04,713 WARNING  [W] too many AD fields at chrX:505629 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,713 WARNING  [W] too many AD fields at chrX:505639 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,714 WARNING  [W] too many AD fields at chrX:505651 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,714 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,714 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,714 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,714 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,715 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,715 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,715 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,716 WARNING  [W] too many AD f

2021-12-16 14:11:04,744 WARNING  [W] too many AD fields at chrX:1166310 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,744 WARNING  [W] too many AD fields at chrX:1168933 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,744 WARNING  [W] too many AD fields at chrX:1168998 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,745 WARNING  [W] too many AD fields at chrX:1169134 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,745 WARNING  [W] too many AD fields at chrX:1169210 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,745 WARNING  [W] too many AD fields at chrX:1169248 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,746 WARNING  [W] too many AD fields at chrX:1169269 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,746 WARNING  [W] too many AD fields at chrX:1169277 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,747 WARNING  [W] too many AD fields at chrX:1169527 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,747 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,747 WARNING  [W] too

2021-12-16 14:11:04,771 WARNING  [W] too many AD fields at chrX:1669029 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,771 WARNING  [W] too many AD fields at chrX:1669050 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,771 WARNING  [W] too many AD fields at chrX:1669089 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,771 WARNING  [W] too many AD fields at chrX:1669097 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,772 WARNING  [W] too many AD fields at chrX:1669142 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,772 WARNING  [W] too many AD fields at chrX:1678251 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,773 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,773 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,773 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,774 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2021-12-16 14:11:04,775 WARNING  [W] too

2021-12-16 14:11:05,567 WARNING  [W] too many AD fields at chrX:2137420 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137421 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137422 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137423 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,568 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,569 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,569 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,569 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,569 WARNING  [W] too

2021-12-16 14:11:05,585 WARNING  [W] too many AD fields at chrX:2228072 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,585 WARNING  [W] too many AD fields at chrX:2228189 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,586 WARNING  [W] too many AD fields at chrX:2228218 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,586 WARNING  [W] too many AD fields at chrX:2243787 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,586 WARNING  [W] too many AD fields at chrX:2276896 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,586 WARNING  [W] too many AD fields at chrX:2277153 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,587 WARNING  [W] too many AD fields at chrX:2277248 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,587 WARNING  [W] too many AD fields at chrX:2277298 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,587 WARNING  [W] too many AD fields at chrX:2277341 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,587 WARNING  [W] too many AD fields at chrX:2278921 max_ad = 2 retrieved: 3
2021-12-16 14:11:05,587 WARNING  [W] too

In [7]:
cp -p  ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/non-targeted/HG002-HifiDV-against-T2Tv2.1XY-halfcalltohomvar-male.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/
cp -p ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.12/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.1XY-halfcalltohomvar-male.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.12/

### <font color='blue'>Step 3: SV Benchmarking T2T-v2.1-XY-align2-GRCh37.dip.vcf agains SV v0.6 GIAB benchmark with truvari</font>
1. Truth = GRCh37 v0.6 SV benchmark -- using GRCh37 because only one with X and Y, Query = T2T-v2.1-XY-align2-GRCh37.dip.vcf

In [9]:
#Get Benchmark callset
wget -P truvari/resources https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.vcf.gz
wget -P truvari/resources https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.bed

--2022-01-04 09:08:03--  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.vcf.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 2607:f220:41f:250::228, 2607:f220:41e:250::7, 130.14.250.13, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|2607:f220:41f:250::228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25738211 (25M) [application/x-gzip]
Saving to: ‘truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz’

HG002_SVs_Tier1_v0. 100%[===================>]  24.54M  11.9MB/s    in 2.1s    

2022-01-04 09:08:05 (11.9 MB/s) - ‘truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz’ saved [25738211/25738211]

--2022-01-04 09:08:05--  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.bed
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 2607:f220:41f:250::

In [10]:
wget -P truvari/resources https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.vcf.gz.tbi

--2022-01-04 10:14:41--  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NIST_SV_v0.6/HG002_SVs_Tier1_v0.6.vcf.gz.tbi
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 2607:f220:41f:250::228, 2607:f220:41e:250::7, 165.112.9.229, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|2607:f220:41f:250::228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458653 (448K) [application/x-gzip]
Saving to: ‘truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz.tbi’

HG002_SVs_Tier1_v0. 100%[===================>] 447.90K   339KB/s    in 1.3s    

2022-01-04 10:14:43 (339 KB/s) - ‘truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz.tbi’ saved [458653/458653]



In [7]:
#run from conda samtools_env.  Dipcall output multi-allelic variants need to first split these. 
bcftools --version
date

bcftools norm -m - dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37.dip.vcf.gz -Oz -o dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37_split_multiallelic.dip.vcf.gz
tabix -p vcf dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37_split_multiallelic.dip.vcf.gz

bcftools 1.8
Using htslib 1.8
Copyright (C) 2016 Genome Research Ltd.
License Expat: The MIT/Expat license
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Mon Jan  3 12:52:00 EST 2022
Lines   total/split/realigned/skipped:	172998/13278/0/0


In [11]:
#subset bed to only include XY.
grep ^"X\|Y" truvari/resources/HG002_SVs_Tier1_v0.6.bed > truvari/resources/HG002_SVs_Tier1_v0.6_XYonly.bed
wc -l truvari/resources/HG002_SVs_Tier1_v0.6_XYonly.bed

    1282 truvari/resources/HG002_SVs_Tier1_v0.6_XYonly.bed


**setup docker container w/ Truvari in bash terminal using the following commnads**
```
docker run -v $(pwd):/here -it ubuntu bash
apt-get update
apt-get install python3 libz-dev python3-pip gcc make
pip3 install --upgrade pip setuptools wheel
pip3 install Truvari
```
Docker container "here" starts at `/Users/jmcdani/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation`

**Truvari v3.1.0 (this is a new version, updated 12/22, first time I am running this), decided to build docker container (see truvari-docker-build notes)**

**Truvari command run from cmd line within Docker container**
```
docker run -it -v $(pwd):/here truvari:v3.1.0 bench \
-b /here/truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz \
-c /here/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37_split_multiallelic.dip.vcf.gz \
-o /here/truvari/T2T-v2.1-XY-align2-GRCh37-bench-SV0.6_allChroms/ \
--multimatch \
--passonly \
-r 2000 \
-C 2000 \
--includebed /here/truvari/resources/HG002_SVs_Tier1_v0.6.bed \
-f /here/dipcall/references/GRCh37.fa

   "TP-base": 240,
    "TP-call": 240,
    "FP": 21,
    "FN": 9401,
    "precision": 0.9195402298850575,
    "recall": 0.024893683227880926,
    "f1": 0.048475055544334476,
    "base cnt": 9641,
    "call cnt": 261,
    "TP-call_TP-gt": 0,
    "TP-call_FP-gt": 240,
    "TP-base_TP-gt": 0,
    "TP-base_FP-gt": 240,
    "gt_concordance": 0.0
```



```
docker run -it -v $(pwd):/here truvari:v3.1.0 bench \
-b /here/truvari/resources/HG002_SVs_Tier1_v0.6.vcf.gz \
-c /here/dipcall/dipcall-output/T2T-v2.1-XY-align2-GRCh37_split_multiallelic.dip.vcf.gz \
-o /here/truvari/T2T-v2.1-XY-align2-GRCh37-bench-SV0.6_XYonly/ \
--multimatch \
--passonly \
-r 2000 \
-C 2000 \
--includebed /here/truvari/resources/HG002_SVs_Tier1_v0.6_XYonly.bed \
-f /here/dipcall/references/GRCh37.fa

    "TP-base": 240,
    "TP-call": 240,
    "FP": 21,
    "FN": 4,
    "precision": 0.9195402298850575,
    "recall": 0.9836065573770492,
    "f1": 0.9504950495049505,
    "base cnt": 244,
    "call cnt": 261,
    "TP-call_TP-gt": 0,
    "TP-call_FP-gt": 240,
    "TP-base_TP-gt": 0,
    "TP-base_FP-gt": 240,
    "gt_concordance": 0.0
```

**NOTES FROM RUNNING TRUVARI**

1/4/22 intially started docker and installed truvari however found that I got error when I run truvari
    `--chunksize must be >= --refdist` We define `--refdist` = 2000.  We don't define `--chunksize` but looking at the code it appears to be default of 1000. I'm not sure what default `--chunksize` was before.  Looking at past logs the default `--chunksize` isn't reported like other default values are but it is now, possible this is a new option.  I checked the Truvari Github and  truvari was updated on 12/22 with changes to "bench" https://github.com/spiralgenetics/truvari/wiki/Updates
    
Ended up testing truvari two ways 1) both -r and -C set to 2000 and 2) -r (500) and -C (1000)run with defaults. JZ noted that option 1 resulted the outputs he was looking for. He also requested subsetting benchmark bed to just X and Y.  

### Truvari Results
JZ noted that he looked at few of the FPs and FNs, and it looks like all of them are either differences in representation or errors in v0.6, so i think the assembly-based calls look good

# <font color='red'>T2T HG002 chrXY v2.5 assembly</font>

**assembly v2.5 from**\ 
https://s3-us-west-2.amazonaws.com/human-pangenomics/working/T2T/HG002XY/v2.1/v2.5.XY.fasta

### <font color='blue'>Step 1: Prepare T2T chrXY assembly for use as benchmark</font>
1. split assembly by chromosome
2. call variants 

In [2]:
#split XY fasta to chrX and chrY
awk '/^>/{f="chr."++d} {print > f}' T2T-chrXY-assembly/v2.5_XY/v2.5.XY.fasta
mv chr.1 T2T-chrXY-assembly/v2.5_XY/chrX_v2.5.fa
mv chr.2 T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa

#check that split fasta have same total lines as complete fasta. Confirm headers are correct. 
wc -l T2T-chrXY-assembly/v2.5_XY/chrX_v2.5.fa
wc -l T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa
wc -l T2T-chrXY-assembly/v2.5_XY/v2.5.XY.fasta
grep '>' T2T-chrXY-assembly/v2.5_XY/chrX_v2.5.fa
grep '>' T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa

 2572498 T2T-chrXY-assembly/v2.5_XY/chrX_v2.5.fa
 1041001 T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa
 3613499 T2T-chrXY-assembly/v2.5_XY/v2.5.XY.fasta
>chrX_hg002
>chrY_hg002


In [4]:
#Run Dipcall v0.3 in Docker container aligning to GRCh38. 
caffeinate -s sh dipcall/dipcall-T2T-chrXY.sh

Tue Jan 11 10:36:37 EST 2022
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa 2> /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap1.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap1.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.5_XY/chrX_v2.5.fa 2> /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap2.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap2.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -a -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.5_XY/chrY_v2.5.fa 2> /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap1.sam.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.hap1.sam.gz
/data/dipcall/dipcall

### <font color='blue'>Step 2 : Small Variant Benchmarking against T2T-v2.5-XY-align2-GRCh38.dip.vcf with hap.py</font>
- run 1_v2.5 Query = HG002-cur.20211005 (HPRC) -- target regions = HG002-cur.20211005.dip.bed
- run 2_v2.5 Query = Hifi-DV (from Google) -- no target regions, Ajust dipcall vcf to normalize GT to match DV GT representations.

#### Hap.py Run #1_v2.5 HG002-cur.20211005 (HPRC) -- target regions = HG002-cur.20211005.dip.bed
run hap.py benchmarking in v3.15 docker container from shell script.  

**Results:**
JZ notes these look good.

In [14]:
# query = HPRC HG002-cur.20211005
caffeinate -s sh happy/happy.sh

start targeted hap.py v3.15 for HPRC HG002-cur.20211005 on date
2022-01-11 17:27:32,995 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         219898
[I] Non-reference VCF records: 168320
[W] overlapping records at chr1:977279 for sample 0
[W] Variants that overlap on the reference allele: 1219
[I] Total VCF records:         5820577
[I] Non-reference VCF records: 5367388
2022-01-11 17:28:21,836 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,836 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,836 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,837 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2022-01-11 

2022-01-11 17:28:21,846 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:515818 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:515929 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD fields at chrX:522301 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,847 WARNING  [W] too many AD f

2022-01-11 17:28:21,857 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,857 WARNING  [W] too many AD fields at chrX:1170088 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,857 WARNING  [W] too many AD fields at chrX:1174060 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,857 WARNING  [W] too many AD fields at chrX:1176328 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1178157 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1178158 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1178184 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1236199 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1236212 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too many AD fields at chrX:1236218 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,858 WARNING  [W] too

2022-01-11 17:28:21,867 WARNING  [W] too many AD fields at chrX:1682200 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,867 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,867 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,867 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,867 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too many AD fields at chrX:1763599 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too many AD fields at chrX:1763601 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too many AD fields at chrX:1763704 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too many AD fields at chrX:1763744 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too many AD fields at chrX:1763750 max_ad = 2 retrieved: 3
2022-01-11 17:28:21,868 WARNING  [W] too

2022-01-11 17:28:22,954 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,955 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,955 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,955 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,955 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too many AD fields at chrX:2137430 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too many AD fields at chrX:2137431 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too many AD fields at chrX:2137432 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too many AD fields at chrX:2137433 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,956 WARNING  [W] too

2022-01-11 17:28:22,990 WARNING  [W] too many AD fields at chrX:2277248 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,990 WARNING  [W] too many AD fields at chrX:2277298 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,990 WARNING  [W] too many AD fields at chrX:2277341 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,991 WARNING  [W] too many AD fields at chrX:2278921 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,991 WARNING  [W] too many AD fields at chrX:2278932 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,991 WARNING  [W] too many AD fields at chrX:2278992 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,991 WARNING  [W] too many AD fields at chrX:2280109 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,992 WARNING  [W] too many AD fields at chrX:2280509 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,992 WARNING  [W] too many AD fields at chrX:2282282 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,992 WARNING  [W] too many AD fields at chrX:2282295 max_ad = 2 retrieved: 3
2022-01-11 17:28:22,993 WARNING  [W] too

**Ran each extended.csv through Rscript `happy/summary_metrics/summary_metrics-benchmarking_against_T2T-vlatest-XY.R` to generate summary metrics. All metrics combined in excel sheet "3.15_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv"**

In [16]:
#copy happy extended.csv files to summary_metrics for R
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.15/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.5XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.15/

#### Hap.py Run #2_v2.5 Query = Hifi-DV (from Google) -- no target regions, Ajust dipcall vcf to normalize GT to match DV GT representations.
DV represents GT differently than dipcall so need to run with pre-processed T2T vcf.
run hap.py benchmarking in v3.15 docker container from shell script.  

**Results:**
JZ notes these look good.  

In [21]:
# adjust GT representation for consistency with query HiFi-DV callset
gunzip -c dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.dip.vcf.gz | sed 's/\.|1/1\/1/;s/1|\./1\/1/' | bgzip -c > dipcall/dipcall-output/T2T-v2.5-XY-align2-GRCh38.dip_halfcalltohomvar.vcf.gz

In [22]:
caffeinate -s sh happy/happy.sh 

start non-targeted hap.py v3.15 HG002-HifiDV on date
2022-01-11 18:39:53,751 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         219898
[I] Non-reference VCF records: 168320
[W] overlapping records at chr1:1077299 for sample 0
[W] Variants that overlap on the reference allele: 183
[I] Total VCF records:         5351977
[I] Non-reference VCF records: 5349601
2022-01-11 18:41:06,814 WARNING  Creating template for vcfeval. You can speed this up by supplying a SDF template that corresponds to /data/dipcall/references/GRCh38.fa
Benchmarking Summary:
Type Filter  TRUTH.TOTAL  TRUTH.TP  TRUTH.FN  QUERY.TOTAL  QUERY.FP  QUERY.UNK  FP.gt  FP.al  METRIC.Recall  METRIC.Precision  METRIC.Frac_NA  METRIC.F1_Score  TRUTH.TOTAL.TiTv_ratio  QUERY.TOTAL.TiTv_ratio  TRU

In [23]:
cp happy/happy-results-3.15/non-targeted/HG002-HifiDV-against-T2Tv2.5XY-halfcalltohomvar.extended.csv happy/summary_metrics/ext_csvs/3.15/

# <font color='red'>T2T HG002 chrXY v2.7 assembly</font>

**assembly 2.7 from**\
https://s3-us-west-2.amazonaws.com/human-pangenomics/working/T2T/HG002XY/v2.1/v2.7.XY.fasta

### <font color='blue'>Step 1: Prepare T2T chrXY assembly for use as benchmark</font>
1. split assembly by chromosome
2. call variants 

In [2]:
#split XY fasta to chrX and chrY
awk '/^>/{f="chr."++d} {print > f}' T2T-chrXY-assembly/v2.7_XY/v2.7.XY.fasta
mv chr.1 T2T-chrXY-assembly/v2.7_XY/chrX_v2.7.fa
mv chr.2 T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa

#check that split fasta have same total lines as complete fasta. Confirm headers are correct. 
wc -l T2T-chrXY-assembly/v2.7_XY/chrX_v2.7.fa
wc -l T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa
wc -l T2T-chrXY-assembly/v2.7_XY/v2.7.XY.fasta
grep '>' T2T-chrXY-assembly/v2.7_XY/chrX_v2.7.fa
grep '>' T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa

 2572498 T2T-chrXY-assembly/v2.7_XY/chrX_v2.7.fa
 1041002 T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa
 3613500 T2T-chrXY-assembly/v2.7_XY/v2.7.XY.fasta
>chrX_hg002
>chrY_hg002


In [5]:
#Run Dipcall v0.3 in Docker container aligning v2.7 to GRCh38. 
caffeinate -s sh dipcall/dipcall-T2T-chrXY.sh

Wed Jan 19 15:29:47 EST 2022
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa 2> /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap1.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap1.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -c --paf-no-hit -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.7_XY/chrX_v2.7.fa 2> /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap2.paf.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap2.paf.gz
/data/dipcall/dipcall.kit_0.3/minimap2 -a -xasm5 --cs -t8 -z200000,10000,200 /data/dipcall/references/GRCh38.fa /data/T2T-chrXY-assembly/v2.7_XY/chrY_v2.7.fa 2> /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap1.sam.gz.log | gzip > /data/dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.hap1.sam.gz
/data/dipcall/dipcall

### <font color='blue'>Step 2 : Small Variant Benchmarking against T2T-v2.7-XY-align2-GRCh38.dip.vcf with hap.py</font>
- run 1_v2.7 Query = HG002-cur.20211005 (HPRC) -- target regions = HG002-cur.20211005.dip.bed
- run 2_v2.7 Query = Hifi-DV (from Google) -- no target regions, Ajust dipcall vcf to normalize GT to match DV GT representations.

#### Hap.py Run #1_v2.7 HG002-cur.20211005 (HPRC) -- target regions = HG002-cur.20211005.dip.bed
run hap.py benchmarking in v3.15 docker container from shell script.  

**Results:**\
similar to v2.5 as expected

In [7]:
#run 1_2.7
caffeinate -s sh happy/happy.sh 

start targeted hap.py v3.15 for HPRC HG002-cur.20211005 on date
2022-01-20 13:19:49,008 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         219881
[I] Non-reference VCF records: 168318
[W] overlapping records at chr1:977279 for sample 0
[W] Variants that overlap on the reference allele: 1219
[I] Total VCF records:         5820577
[I] Non-reference VCF records: 5367388
2022-01-20 13:20:38,066 WARNING  [W] too many AD fields at chrX:250327 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,067 WARNING  [W] too many AD fields at chrX:250352 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,067 WARNING  [W] too many AD fields at chrX:250376 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,067 WARNING  [W] too many AD fields at chrX:250402 max_ad = 2 retrieved: 3
2022-01-20 

2022-01-20 13:20:38,076 WARNING  [W] too many AD fields at chrX:505674 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,076 WARNING  [W] too many AD fields at chrX:505700 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,076 WARNING  [W] too many AD fields at chrX:505895 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,076 WARNING  [W] too many AD fields at chrX:505939 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:515797 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:515816 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:515817 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:515818 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:515929 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD fields at chrX:522301 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,077 WARNING  [W] too many AD f

2022-01-20 13:20:38,088 WARNING  [W] too many AD fields at chrX:1169597 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,088 WARNING  [W] too many AD fields at chrX:1170088 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,088 WARNING  [W] too many AD fields at chrX:1174060 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,088 WARNING  [W] too many AD fields at chrX:1176328 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,088 WARNING  [W] too many AD fields at chrX:1178157 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too many AD fields at chrX:1178158 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too many AD fields at chrX:1178184 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too many AD fields at chrX:1236199 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too many AD fields at chrX:1236212 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too many AD fields at chrX:1236218 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,089 WARNING  [W] too

2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1682200 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1735304 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1762451 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1763582 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1763598 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,103 WARNING  [W] too many AD fields at chrX:1763599 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,104 WARNING  [W] too many AD fields at chrX:1763601 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,109 WARNING  [W] too many AD fields at chrX:1763704 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,109 WARNING  [W] too many AD fields at chrX:1763744 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,109 WARNING  [W] too many AD fields at chrX:1763750 max_ad = 2 retrieved: 3
2022-01-20 13:20:38,109 WARNING  [W] too

2022-01-20 13:20:39,114 WARNING  [W] too many AD fields at chrX:2137424 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,115 WARNING  [W] too many AD fields at chrX:2137425 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,115 WARNING  [W] too many AD fields at chrX:2137426 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,115 WARNING  [W] too many AD fields at chrX:2137427 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,116 WARNING  [W] too many AD fields at chrX:2137428 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,116 WARNING  [W] too many AD fields at chrX:2137429 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,117 WARNING  [W] too many AD fields at chrX:2137430 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,117 WARNING  [W] too many AD fields at chrX:2137431 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,117 WARNING  [W] too many AD fields at chrX:2137432 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,117 WARNING  [W] too many AD fields at chrX:2137433 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,118 WARNING  [W] too

2022-01-20 13:20:39,151 WARNING  [W] too many AD fields at chrX:2277248 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,151 WARNING  [W] too many AD fields at chrX:2277298 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,152 WARNING  [W] too many AD fields at chrX:2277341 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,152 WARNING  [W] too many AD fields at chrX:2278921 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,153 WARNING  [W] too many AD fields at chrX:2278932 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,153 WARNING  [W] too many AD fields at chrX:2278992 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,153 WARNING  [W] too many AD fields at chrX:2280109 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,153 WARNING  [W] too many AD fields at chrX:2280509 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,154 WARNING  [W] too many AD fields at chrX:2282282 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,154 WARNING  [W] too many AD fields at chrX:2282295 max_ad = 2 retrieved: 3
2022-01-20 13:20:39,154 WARNING  [W] too

#### Hap.py Run #2_v2.7 Query = Hifi-DV (from Google) -- no target regions, Ajust dipcall vcf to normalize GT to match DV GT representations.
DV represents GT differently than dipcall so need to run with pre-processed T2T vcf.
run hap.py benchmarking in v3.15 docker container from shell script.  

**Results:**\
similar to v2.5 as expected

In [8]:
# adjust GT representation for consistency with query HiFi-DV callset
gunzip -c dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.dip.vcf.gz | sed 's/\.|1/1\/1/;s/1|\./1\/1/' | bgzip -c > dipcall/dipcall-output/T2T-v2.7-XY-align2-GRCh38.dip_halfcalltohomvar.vcf.gz

In [10]:
#run 2_2.7
caffeinate -s sh happy/happy.sh 

start non-targeted hap.py v3.15 HG002-HifiDV on date
2022-01-20 13:43:09,054 WARNING  No reference file found at default locations. You can set the environment variable 'HGREF' or 'HG19' to point to a suitable Fasta file.
Hap.py 
[W] overlapping records at chrX:1042332 for sample 0
[W] Variants that overlap on the reference allele: 6
[I] Total VCF records:         219881
[I] Non-reference VCF records: 168318
[W] overlapping records at chr1:1077299 for sample 0
[W] Variants that overlap on the reference allele: 183
[I] Total VCF records:         5351977
[I] Non-reference VCF records: 5349601
2022-01-20 13:44:23,526 WARNING  Creating template for vcfeval. You can speed this up by supplying a SDF template that corresponds to /data/dipcall/references/GRCh38.fa
Benchmarking Summary:
Type Filter  TRUTH.TOTAL  TRUTH.TP  TRUTH.FN  QUERY.TOTAL  QUERY.FP  QUERY.UNK  FP.gt  FP.al  METRIC.Recall  METRIC.Precision  METRIC.Frac_NA  METRIC.F1_Score  TRUTH.TOTAL.TiTv_ratio  QUERY.TOTAL.TiTv_ratio  TRU

**Ran each extended.csv through Rscript `happy/summary_metrics/summary_metrics-benchmarking_against_T2T-vlatest-XY.R` to generate summary metrics. All metrics combined in excel sheet "3.15_benchmarking_summary_metrics-against-T2T-v2.1-XY.csv"**

In [13]:
#copy happy extended.csv files to summary_metrics for R
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.15/targeted/HPRC-HG002-cur.20211005-against-T2Tv2.7XY.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.15/
cp ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/happy-results-3.15/non-targeted/HG002-HifiDV-against-T2Tv2.5XY-halfcalltohomvar.extended.csv ~/Documents/GiaB/Benchmarking/assembly-benchmark-development/chrXY_assembly-evaluation/happy/summary_metrics/ext_csvs/3.15/